# Calculate Price Elasticity of Demand

In [37]:
import pandas as pd
import statsmodels.api as sm 
import numpy as np

In [3]:
final_sell_calendar_households = pd.read_csv('final_sell_calendar_households.csv')

In [4]:
final_sell_calendar_hobbies = pd.read_csv('final_sell_calendar_hobbies.csv')

In [5]:
final_sell_calendar_foods = pd.read_csv('final_sell_calendar_foods.csv')

In [35]:
def calculate_ped(group):
    X = sm.add_constant(group['sell_price'])
    model = sm.OLS(group['sold'], X).fit()
    price_coef = model.params['sell_price']
    mean_sellprice = np.mean(group['sell_price'])
    mean_quantity = np.mean(group['sold'])
    ped = price_coef * (mean_sellprice / mean_quantity)
    return ped

## Calculate PED for hobbies items

In [8]:
final_sell_calendar_hobbies =final_sell_calendar_hobbies.drop(final_sell_calendar_hobbies.columns[0], axis =1)

In [9]:
final_sell_calendar_hobbies

,id,wm_yr_wk,sell_price,d,year,month,wday,date,amount,number
0,HOBBIES_1_001_CA_1,11325,9.58,d_897,2013,7,1,2013-07-13,0,897
1,HOBBIES_1_001_CA_1,11325,9.58,d_898,2013,7,2,2013-07-14,0,898
2,HOBBIES_1_001_CA_1,11325,9.58,d_899,2013,7,3,2013-07-15,0,899
3,HOBBIES_1_001_CA_1,11325,9.58,d_900,2013,7,4,2013-07-16,0,900
4,HOBBIES_1_001_CA_1,11325,9.58,d_901,2013,7,5,2013-07-17,0,901
...,...,...,...,...,...,...,...,...,...,...
8642680,HOBBIES_2_148_WI_3,11105,0.97,d_31,2011,2,3,2011-02-28,0,31
8642681,HOBBIES_2_148_WI_3,11105,0.97,d_32,2011,3,4,2011-03-01,0,32
8642682,HOBBIES_2_148_WI_3,11105,0.97,d_33,2011,3,5,2011-03-02,0,33
8642683,HOBBIES_2_148_WI_3,11105,0.97,d_34,2011,3,6,2011-03-03,0,34


In [10]:
final_sell_calendar_hobbies[['cat_id','dept_id','item_id', 'state_id','store_id']] = final_sell_calendar_hobbies['id'].str.split('_', expand=True)

In [15]:
ped_id = final_sell_calendar_hobbies.groupby('id').apply(calculate_ped)
print(ped_id)

/Users/chloeang/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


PED by id
id
HOBBIES_1_001_CA_1    -5.097742
HOBBIES_1_001_CA_2    35.232799
HOBBIES_1_001_CA_3    34.909486
HOBBIES_1_001_CA_4    -4.440415
HOBBIES_1_001_TX_1     0.833142
                        ...    
HOBBIES_2_149_TX_2    -1.109734
HOBBIES_2_149_TX_3    -0.940017
HOBBIES_2_149_WI_1    -1.222426
HOBBIES_2_149_WI_2    -0.790498
HOBBIES_2_149_WI_3    -1.312173
Length: 5650, dtype: float64


In [17]:
ped_item_id = final_sell_calendar_hobbies.groupby('item_id').apply(calculate_ped)
ped_item_id = ped_item_id.to_frame()
ped_item_id = ped_item_id.reset_index()

PED by item id
item_id
001    3.978094
002    0.376649
003   -1.100386
004    2.916829
005   -3.049954
         ...   
420    0.012902
421   -2.901577
422   -2.203539
423   -0.878666
424    1.420512
Length: 420, dtype: float64


In [22]:
ped_item_dept_id = final_sell_calendar_hobbies.groupby(['item_id','dept_id']).apply(calculate_ped)
ped_item_dept_id = ped_item_dept_id.to_frame()
ped_item_dept_id =ped_item_dept_id.reset_index()

PED by item id and dept id
item_id  dept_id
001      1         -6.313660
         2         -0.151866
002      1          1.685968
         2          0.218097
003      1         -0.221408
                      ...   
420      1          0.012902
421      1         -2.901577
422      1         -2.203539
423      1         -0.878666
424      1          1.420512
Length: 565, dtype: float64


In [40]:
ped_item_dept_state_id = final_sell_calendar_hobbies.groupby(['item_id','dept_id','state_id']).apply(calculate_ped)
ped_item_dept_state_id = ped_item_dept_state_id.to_frame()
ped_item_dept_state_id = ped_item_dept_state_id.reset_index()

/Users/chloeang/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [47]:
ped_item_dept_state_store_id = final_sell_calendar_hobbies.groupby(['item_id','dept_id','state_id','store_id']).apply(calculate_ped)
ped_item_dept_state_store_id = ped_item_dept_state_store_id.to_frame()
ped_item_dept_state_store_id = ped_item_dept_state_store_id.reset_index()

/Users/chloeang/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [85]:
a = ped_item_id.merge(ped_item_dept_id,left_on = 'item_id', right_on='item_id')
a

,item_id,0_x,dept_id,0_y
0,001,3.978094,1,-6.313660
1,001,3.978094,2,-0.151866
2,002,0.376649,1,1.685968
3,002,0.376649,2,0.218097
4,003,-1.100386,1,-0.221408
...,...,...,...,...
560,420,0.012902,1,0.012902
561,421,-2.901577,1,-2.901577
562,422,-2.203539,1,-2.203539
563,423,-0.878666,1,-0.878666


In [86]:
b = a.merge(ped_item_dept_state_id,left_on =['item_id','dept_id'],right_on =['item_id','dept_id'])

,item_id,0_x,dept_id,0_y,state_id,0
0,001,3.978094,1,-6.313660,CA,-4.384036
1,001,3.978094,1,-6.313660,TX,0.174109
2,001,3.978094,1,-6.313660,WI,-10.304710
3,001,3.978094,2,-0.151866,CA,-1.848419
4,001,3.978094,2,-0.151866,TX,0.259439
...,...,...,...,...,...,...
1690,423,-0.878666,1,-0.878666,TX,2.607444
1691,423,-0.878666,1,-0.878666,WI,-2.425794
1692,424,1.420512,1,1.420512,CA,-2.579196
1693,424,1.420512,1,1.420512,TX,-2.743452


In [87]:
c = b.merge(ped_item_dept_state_store_id,left_on =['item_id','dept_id','state_id'],right_on =['item_id','dept_id','state_id'])
c

/Users/chloeang/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'0_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(


,item_id,0_x,dept_id,0_y,state_id,0_x,store_id,0_y
0,001,3.978094,1,-6.313660,CA,-4.384036,1,-5.097742
1,001,3.978094,1,-6.313660,CA,-4.384036,2,35.232799
2,001,3.978094,1,-6.313660,CA,-4.384036,3,34.909486
3,001,3.978094,1,-6.313660,CA,-4.384036,4,-4.440415
4,001,3.978094,1,-6.313660,TX,0.174109,1,0.833142
...,...,...,...,...,...,...,...,...
5645,424,1.420512,1,1.420512,TX,-2.743452,2,-1.997759
5646,424,1.420512,1,1.420512,TX,-2.743452,3,-0.580792
5647,424,1.420512,1,1.420512,WI,-3.536496,1,-3.177566
5648,424,1.420512,1,1.420512,WI,-3.536496,2,-2.644528


In [88]:
hobbies_ped = c.rename(columns={"0_x": "a", "0_y": "b", "0_y": "c"})

In [90]:
hobbies_ped.to_csv('hobbies_ped.csv')

## Calculate PED for foods items

In [20]:
foods_df = pd.read_csv('final_sell_calendar_foods2.csv')

In [22]:
foods_df =foods_df.drop(foods_df.columns[0],axis=1)

In [29]:
foods_df_2 = foods_df.dropna()

In [31]:
foods_df_2.reset_index(inplace= True, drop = True)

In [32]:
foods_df_2

,id,item_id,dept_id,cat_id,store_id,date,month,year,wm_yr_wk,sell_price,sold
0,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,2011-01-29,1,2011,11101,2.00,3
1,FOODS_1_002_CA_1_validation,FOODS_1_002,FOODS_1,FOODS,CA_1,2011-01-29,1,2011,11101,7.88,0
2,FOODS_1_003_CA_1_validation,FOODS_1_003,FOODS_1,FOODS,CA_1,2011-01-29,1,2011,11101,2.88,0
3,FOODS_1_005_CA_1_validation,FOODS_1_005,FOODS_1,FOODS,CA_1,2011-01-29,1,2011,11101,2.94,3
4,FOODS_1_006_CA_1_validation,FOODS_1_006,FOODS_1,FOODS,CA_1,2011-01-29,1,2011,11101,1.97,0
...,...,...,...,...,...,...,...,...,...,...,...
21395948,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,2016-04-24,4,2016,11613,2.98,1
21395949,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,2016-04-24,4,2016,11613,2.48,0
21395950,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,2016-04-24,4,2016,11613,3.98,0
21395951,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,2016-04-24,4,2016,11613,1.28,3


In [25]:
na

,id,item_id,dept_id,cat_id,store_id,date,month,year,wm_yr_wk,sell_price,sold


In [39]:
ped_id_food =foods_df_2.groupby('id').apply(calculate_ped)
print(ped_id_food)

id
FOODS_1_001_CA_1_validation   -3.395525
FOODS_1_001_CA_2_validation   -4.022723
FOODS_1_001_CA_3_validation   -1.805704
FOODS_1_001_CA_4_validation   -2.857772
FOODS_1_001_TX_1_validation   -1.110851
                                 ...   
FOODS_3_827_TX_2_validation    1.000000
FOODS_3_827_TX_3_validation   -3.714840
FOODS_3_827_WI_1_validation    1.000000
FOODS_3_827_WI_2_validation    1.000000
FOODS_3_827_WI_3_validation    1.000000
Length: 14370, dtype: float64


In [40]:
ped_item_id_food =foods_df_2.groupby('item_id').apply(calculate_ped)
print(ped_item_id_food)

item_id
FOODS_1_001   -3.243099
FOODS_1_002    0.351640
FOODS_1_003   -3.270178
FOODS_1_004   -3.543704
FOODS_1_005    1.188883
                 ...   
FOODS_3_823   -2.578088
FOODS_3_824    1.205180
FOODS_3_825   -5.846464
FOODS_3_826    3.399996
FOODS_3_827   -1.053623
Length: 1437, dtype: float64


In [ ]:
ped_item_id = final_sell_calendar_foods.groupby('item_id').apply(calculate_ped)
ped_item_id = ped_item_id.to_frame()
ped_item_id = ped_item_id.reset_index()

In [ ]:
ped_item_dept_id = final_sell_calendar_foods.groupby(['item_id','dept_id']).apply(calculate_ped)
ped_item_dept_id = ped_item_dept_id.to_frame()
ped_item_dept_id =ped_item_dept_id.reset_index()

In [ ]:
ped_item_dept_state_id = final_sell_calendar_foods.groupby(['item_id','dept_id','state_id']).apply(calculate_ped)
ped_item_dept_state_id = ped_item_dept_state_id.to_frame()
ped_item_dept_state_id = ped_item_dept_state_id.reset_index()

In [ ]:
ped_item_dept_state_store_id = final_sell_calendar_foods.groupby(['item_id','dept_id','state_id','store_id']).apply(calculate_ped)
ped_item_dept_state_store_id = ped_item_dept_state_store_id.to_frame()
ped_item_dept_state_store_id = ped_item_dept_state_store_id.reset_index()

In [ ]:
a = ped_item_id.merge(ped_item_dept_id,left_on = 'item_id', right_on='item_id')
a

In [116]:
final_sell_calendar_foods

,id,wm_yr_wk,sell_price,d,year,month,wday,date,amount,number,cat_id,dept_id
0,FOODS_1_001_CA_1,11325,2.24,d_897,2013,7,1,2013-07-13,2,897,FOODS,1
1,FOODS_1_001_CA_1,11325,2.24,d_898,2013,7,2,2013-07-14,0,898,FOODS,1
2,FOODS_1_001_CA_1,11325,2.24,d_899,2013,7,3,2013-07-15,1,899,FOODS,1
3,FOODS_1_001_CA_1,11325,2.24,d_900,2013,7,4,2013-07-16,3,900,FOODS,1
4,FOODS_1_001_CA_1,11325,2.24,d_901,2013,7,5,2013-07-17,0,901,FOODS,1
...,...,...,...,...,...,...,...,...,...,...,...,...
21395948,FOODS_3_825_WI_3,11105,4.00,d_31,2011,2,3,2011-02-28,1,31,FOODS,3
21395949,FOODS_3_825_WI_3,11105,4.00,d_32,2011,3,4,2011-03-01,2,32,FOODS,3
21395950,FOODS_3_825_WI_3,11105,4.00,d_33,2011,3,5,2011-03-02,2,33,FOODS,3
21395951,FOODS_3_825_WI_3,11105,4.00,d_34,2011,3,6,2011-03-03,2,34,FOODS,3


## Caculate PED for households items

In [102]:
final_sell_calendar_households =final_sell_calendar_households.drop(final_sell_calendar_households.columns[0],axis=1)

In [104]:
final_sell_calendar_households[['cat_id','dept_id','item_id', 'state_id','store_id']] = final_sell_calendar_households['id'].str.split('_', expand=True)

In [106]:
final_sell_calendar_households.to_csv('final_sell_calendar_households_2.csv')

In [109]:
final_sell_calendar_households 

,id,wm_yr_wk,sell_price,d,year,month,wday,date,amount,number,cat_id,dept_id,item_id,state_id,store_id
0,HOUSEHOLD_1_001_CA_1,11325,5.97,d_897,2013,7,1,2013-07-13,0,897,HOUSEHOLD,1,001,CA,1
1,HOUSEHOLD_1_001_CA_1,11325,5.97,d_898,2013,7,2,2013-07-14,0,898,HOUSEHOLD,1,001,CA,1
2,HOUSEHOLD_1_001_CA_1,11325,5.97,d_899,2013,7,3,2013-07-15,0,899,HOUSEHOLD,1,001,CA,1
3,HOUSEHOLD_1_001_CA_1,11325,5.97,d_900,2013,7,4,2013-07-16,0,900,HOUSEHOLD,1,001,CA,1
4,HOUSEHOLD_1_001_CA_1,11325,5.97,d_901,2013,7,5,2013-07-17,0,901,HOUSEHOLD,1,001,CA,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15989314,HOUSEHOLD_2_516_WI_3,11105,5.94,d_31,2011,2,3,2011-02-28,0,31,HOUSEHOLD,2,516,WI,3
15989315,HOUSEHOLD_2_516_WI_3,11105,5.94,d_32,2011,3,4,2011-03-01,0,32,HOUSEHOLD,2,516,WI,3
15989316,HOUSEHOLD_2_516_WI_3,11105,5.94,d_33,2011,3,5,2011-03-02,0,33,HOUSEHOLD,2,516,WI,3
15989317,HOUSEHOLD_2_516_WI_3,11105,5.94,d_34,2011,3,6,2011-03-03,0,34,HOUSEHOLD,2,516,WI,3


In [110]:
ped_id_hh = final_sell_calendar_households.groupby('id').apply(calculate_ped)
ped_id_hh = ped_id_hh.to_frame()
ped_id_hh = ped_id_hh.reset_index()

/Users/chloeang/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [111]:
ped_id_hh

,id,0
0,HOUSEHOLD_1_001_CA_1,-0.994784
1,HOUSEHOLD_1_001_CA_2,10.577096
2,HOUSEHOLD_1_001_CA_3,3.494343
3,HOUSEHOLD_1_001_CA_4,0.134957
4,HOUSEHOLD_1_001_TX_1,-2.200022
...,...,...
10465,HOUSEHOLD_2_516_TX_2,-5.833195
10466,HOUSEHOLD_2_516_TX_3,1.000000
10467,HOUSEHOLD_2_516_WI_1,1.000000
10468,HOUSEHOLD_2_516_WI_2,1.000000


In [112]:
ped_item_id_hh = final_sell_calendar_households.groupby(['item_id']).apply(calculate_ped)
ped_item_id_hh = ped_item_id_hh.to_frame()
ped_item_id_hh = ped_item_id_hh.reset_index()

/Users/chloeang/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [113]:
ped_item_id_hh

,item_id,0
0,001,0.561159
1,002,-0.445138
2,003,-4.671168
3,004,-1.487517
4,005,2.282357
...,...,...
536,537,-0.545878
537,538,1.000000
538,539,0.744237
539,540,1.270025


In [118]:
ped_item_dept_id_hh = final_sell_calendar_households.groupby(['item_id','dept_id']).apply(calculate_ped)
ped_item_dept_id_hh = ped_item_dept_id_hh.to_frame()
ped_item_dept_id_hh = ped_item_dept_id_hh.reset_index()

In [119]:
ped_item_dept_id_hh

,item_id,dept_id,0
0,001,1,1.609250
1,001,2,-1.048037
2,002,1,-0.215320
3,002,2,-7.491919
4,003,1,1.000000
...,...,...,...
1042,537,1,-0.545878
1043,538,1,1.000000
1044,539,1,0.744237
1045,540,1,1.270025


In [ ]:
ped_item_dept_id_hh = final_sell_calendar_households.groupby(['item_id','dept_id']).apply(calculate_ped)
ped_item_dept_id_hh = ped_item_dept_id_hh.to_frame()
ped_item_dept_id_hh = ped_item_dept_id_hh.reset_index()